<a href="https://www.kaggle.com/code/darvack/transformer-paper-regression?scriptVersionId=131799847" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/transformer/DatasetB.csv
/kaggle/input/transformer/DatasetA.csv


Here, we have loaded the data and set Furan as the label.
At first, we have used 25 percent of the dataset A as the test set to come up with a good model, and then use this model to test in the dataset B.

In [2]:
ds_A = pd.read_csv("/kaggle/input/transformer/DatasetA.csv")
ds_B = pd.read_csv("/kaggle/input/transformer/DatasetB.csv")

# Splitting train and test
from sklearn.model_selection import train_test_split
train_set_A, test_set_A = train_test_split(ds_A, test_size = 0.2, random_state = 11)

# Setting the labels
y_train_A = train_set_A['Furan']
y_test_A = test_set_A['Furan']

# Dropping the Furan and Health Index columns
X_train_A = train_set_A.drop(["Furan", "HI"], axis = 1)
X_test_A = test_set_A.drop(["Furan", "HI"], axis = 1)

# For Dataset B
y_B = ds_B['Furan']
X_B = ds_B.drop(["Furan", "HI"], axis = 1)

# The code below is for the second case, where we train the data for the whole
# Dataset A and test it on Dataset B
y_A = ds_A['Furan']
X_A = ds_A.drop(["Furan", "HI"], axis = 1)

In [3]:
#ds_A.hist(bins=50, figsize=(20,15))

The code below, drops the columns that we don't need, and only keeps the common features between dataset A and B.

In [4]:
X_train_A = X_train_A.drop(set(ds_A.columns) - set(ds_B.columns), axis=1)
X_test_A = X_test_A.drop(set(ds_A.columns) - set(ds_B.columns), axis=1)
X_A = X_A.drop(set(ds_A.columns) - set(ds_B.columns), axis=1)
X_B = X_B[X_train_A.columns]
X_train_A

,H2,Methane,Acetylene,Ethylene,Ethane,Water,Acid,BDV,IFT
503,19.0,11.90,0.0,2.7,3.0,5,0.005,73.0,30
679,13.2,9.20,0.0,0.6,2.0,4,0.005,88.0,41
240,14.4,2.50,0.0,0.9,1.5,4,0.005,79.0,31
21,13.2,1.10,0.0,1.8,2.1,4,0.024,78.0,25
56,20.9,0.00,0.6,4.2,3.8,6,0.056,76.0,18
...,...,...,...,...,...,...,...,...,...
269,13.7,5.10,0.0,0.4,1.1,1,0.005,94.0,36
337,32.9,3.77,0.0,0.6,2.4,6,0.005,79.0,32
91,22.8,3.30,0.0,4.9,3.0,11,0.140,88.0,16
80,61.2,27.30,0.0,25.6,20.8,9,0.099,70.0,17


The code below performs feature selection via step-wise regression or SequentialFeatureSelector. "n_features_to_select" is a hyperparameter that defines how many feature we'd like to keep which needs to be fine tuned. In our experiment, 5 or 7 features to keep, results in better models. Here we used 7.

In [5]:
'''from sklearn.feature_selection import SequentialFeatureSelector
from catboost import CatBoostRegressor

for i in range(1,9):
    cat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,
                               depth=6, loss_function='RMSE', random_seed=11)

    sfs = SequentialFeatureSelector(cat_reg,
                                    n_features_to_select=i,
                                    direction='forward',
                                    scoring='neg_mean_squared_error',
                                    cv=5)

    sfs.fit(X_train_A, y_train_A)

    # Print the selected features
    print("Selected features" + f"{i}:", sfs.get_feature_names_out())


useful_features1 = ['IFT']
useful_features2 = ['Methane', 'IFT']
useful_features3 = ['Methane', 'Ethane', 'IFT']
useful_features4 = ['Methane', 'Ethane', 'Water', 'IFT']
useful_features5 = ['Methane', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features6 = ['Methane', 'Ethylene', 'Ethane', 'Water', 'BDV', 'IFT']
useful_features7 = ['H2', 'Methane', 'Acetylene', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features8 = ['H2', 'Methane', 'Ethylene', 'Ethane', 'Water', 'Acid', 'BDV', 'IFT']
'''

'from sklearn.feature_selection import SequentialFeatureSelector\nfrom catboost import CatBoostRegressor\n\nfor i in range(1,9):\n    cat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,\n                               depth=6, loss_function=\'RMSE\', random_seed=11)\n\n    sfs = SequentialFeatureSelector(cat_reg,\n                                    n_features_to_select=i,\n                                    direction=\'forward\',\n                                    scoring=\'neg_mean_squared_error\',\n                                    cv=5)\n\n    sfs.fit(X_train_A, y_train_A)\n\n    # Print the selected features\n    print("Selected features" + f"{i}:", sfs.get_feature_names_out())\n\n\nuseful_features1 = [\'IFT\']\nuseful_features2 = [\'Methane\', \'IFT\']\nuseful_features3 = [\'Methane\', \'Ethane\', \'IFT\']\nuseful_features4 = [\'Methane\', \'Ethane\', \'Water\', \'IFT\']\nuseful_features5 = [\'Methane\', \'Ethylene\', \'Ethane\', \'BDV\', \'IFT\']\nus

In [6]:
useful_features1 = ['IFT']
useful_features2 = ['Methane', 'IFT']
useful_features3 = ['Methane', 'Ethane', 'IFT']
useful_features4 = ['Methane', 'Ethane', 'Water', 'IFT']
useful_features5 = ['Methane', 'Water', 'Ethane', 'BDV', 'IFT']
useful_features6 = ['Methane', 'Acetylene', 'Ethane', 'Water', 'BDV', 'IFT']
useful_features7 = ['Water', 'Methane', 'Acetylene', 'Ethylene', 'Ethane', 'BDV', 'IFT']
useful_features8 = ['H2', 'Methane', 'Ethylene', 'Ethane', 'Water', 'Acetylene', 'BDV', 'IFT']

# First case: Training using 80% of the data and testing on the remaining 20

We have experimented a combination of different models in the ensemble.
Although the results were quite similar, we found that a combination of KNN, svm, mlp and logistic regression works best.
In the code below we have created a voting classifier consist of these models.

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import BayesianRidge
from catboost import CatBoostRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

rf_reg = RandomForestRegressor(n_jobs = -1, max_depth = 50)
# svm_reg = SVR(kernel='linear')
# knn_reg = KNeighborsRegressor(n_neighbors=3)
xgb_reg = XGBRegressor(learning_rate=0.01, n_estimators=300, max_depth=3, subsample=0.7)
#mlp_reg = MLPRegressor(hidden_layer_sizes=(100,), max_iter=10000)
ada_reg = AdaBoostRegressor(n_estimators=50, learning_rate=0.003)
#lr_reg = LinearRegression()
#bay_reg = BayesianRidge()
cat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,
                           depth=6, loss_function='RMSE', random_seed=11)
#pls_reg = PLSRegression(n_components=2)
#rig_reg = Ridge(alpha=1.0)
lgb_reg = lgb.LGBMRegressor()
#el_reg = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=0)
'''bag_reg = BaggingRegressor(
    DecisionTreeRegressor(), n_estimators=50, max_samples=100,
    max_features=1.0,
    bootstrap=True,
    n_jobs=-1)'''

voting_reg = VotingRegressor(
  estimators=[#('nn', mlp_reg),
              #('svc', svm_reg),
              #('knn', knn_reg), 
              #('ada', ada_reg),#('by', bay_reg),
              ('xgb', xgb_reg),('cat', cat_reg)
              #('rf', rf_reg),
              #('el', el_reg), ('lgb', lgb_reg)
             ])
voting_reg.fit(X_train_A[useful_features7], y_train_A)

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.01, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=3,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=300, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x7e2aafac7a00>)])

Here is a comparison of different models and the voting classifier.

In [8]:
#from sklearn.linear_model import ElasticNet
#el_reg = ElasticNet(alpha=0.5, l1_ratio=1, random_state=0)

from sklearn.metrics import mean_squared_error
for reg in (#mlp_reg, #svm_reg,
            ada_reg,
            #knn_reg,
            xgb_reg, rf_reg, #bay_reg, el_reg, pls_reg,
            cat_reg,lgb_reg, #bag_reg,lr_reg,
            voting_reg):
    reg.fit(X_train_A[useful_features7], y_train_A)
    y_pred_A = reg.predict(X_test_A[useful_features7])
    y_pred_B = reg.predict(X_B[useful_features7])
    print(reg.__class__.__name__ + " for dataset A:", mean_squared_error(y_test_A, y_pred_A))
    print(reg.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

AdaBoostRegressor for dataset A: 0.5531083657285428
AdaBoostRegressor for dataset B: 1.6662943722224746
XGBRegressor for dataset A: 0.46717819239421476
XGBRegressor for dataset B: 1.692358673806465
RandomForestRegressor for dataset A: 0.493021379379671
RandomForestRegressor for dataset B: 1.6616086089021644
CatBoostRegressor for dataset A: 0.46533980754905646
CatBoostRegressor for dataset B: 1.5282682850173455
LGBMRegressor for dataset A: 0.6064139814318511
LGBMRegressor for dataset B: 1.8836877501508225
VotingRegressor for dataset A: 0.44697177602884747
VotingRegressor for dataset B: 1.5848119720082712


In [9]:
xgb_reg = XGBRegressor(learning_rate=0.01, n_estimators=300, max_depth=3, subsample=0.7)
xgb_reg.fit(X_train_A[useful_features7], np.array(y_train_A).ravel())
y_pred_A = xgb_reg.predict(X_test_A[useful_features7])
y_pred_B = xgb_reg.predict(X_B[useful_features7])
print(xgb_reg.__class__.__name__ + " for dataset A:", mean_squared_error(y_test_A, y_pred_A))
print(xgb_reg.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

XGBRegressor for dataset A: 0.46717819239421476
XGBRegressor for dataset B: 1.692358673806465


In [10]:
cat_reg = CatBoostRegressor(iterations=5000, learning_rate=0.3, verbose = 0,
                           depth=6, loss_function='RMSE', random_seed=11)
cat_reg.fit(X_train_A[useful_features7], y_train_A)
y_pred_A = cat_reg.predict(X_test_A[useful_features7])
y_pred_B = cat_reg.predict(X_B[useful_features7])
print(cat_reg.__class__.__name__ + " for dataset A:", mean_squared_error(y_test_A, y_pred_A))
print(cat_reg.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

CatBoostRegressor for dataset A: 0.4416094676778565
CatBoostRegressor for dataset B: 1.4683711765606906


# Finding the best hyperparameters for CatRegressor

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from optuna import create_study, study
from optuna.integration import CatBoostPruningCallback
import optuna

catboost_reg = CatBoostRegressor()

study = create_study(direction='minimize', pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))

def objective(trial):
    params = {
        #"early_stopping_rounds": 5,
        "loss_function": "RMSE",
        "iterations": 500, #trial.suggest_int('iterations', 250, 800),
        'random_state': 42,
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.7, 100, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'random_strength': trial.suggest_float('random_strength', 0.2, 0.8),
        #'bagging_temperature': trial.suggest_float('bagging_temperature', 5, 22),
        'border_count': 254, # trial.suggest_int('border_count', 20, 60),
        'min_child_samples': trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32]),
        'grow_policy': 'SymmetricTree',
        'use_best_model' : True,
        'eval_metric' : 'RMSE',
        'od_type' : 'Iter',
        'od_wait' : 5,
        'logging_level' : 'Silent',
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True)}

    # Create a new model with the specified parameters
    new_catboost_reg = CatBoostRegressor(**params)
    
    pruning_callback = CatBoostPruningCallback(trial, "RMSE")
    new_catboost_reg.fit(X_train_A[useful_features7], y_train_A,
                         eval_set=[(X_test_A[useful_features7], y_test_A)], verbose=0,
                         early_stopping_rounds=15, callbacks=[pruning_callback])

    # evoke pruning manually.
    pruning_callback.check_pruned()

    # Make predictions on the test set
    y_pred_A = new_catboost_reg.predict(X_test_A[useful_features7])

    # Evaluate the model
    loss_A = mean_squared_error(y_test_A, y_pred_A)

    return loss_A

# Optimize the objective function
study.optimize(objective, n_trials=500, n_jobs=-1, show_progress_bar=True)

# Print the best parameters
best_params = study.best_params
print(best_params)

# Fit the model to the data
new_catboost_reg = CatBoostRegressor(**best_params)
new_catboost_reg.fit(X_train_A[useful_features7], y_train_A, eval_set=[(X_test_A[useful_features7], y_test_A)], verbose=0)

# Make predictions on the test set
y_pred_A = new_catboost_reg.predict(X_test_A[useful_features7])
y_pred_B = new_catboost_reg.predict(X_B[useful_features7])

# Evaluate the model
loss_A = mean_squared_error(y_test_A, y_pred_A)
loss_B = mean_squared_error(y_B, y_pred_B)
print('MSE:', f" loss_A: {loss_A}")
print('MSE:', f" loss_B: {loss_B}")

[I 2023-06-01 06:06:16,032] A new study created in memory with name: no-name-00464be3-c8cc-4651-9cc2-5843d1ceb182
/opt/conda/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/500 [00:00<?, ?it/s]

/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")
/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


[I 2023-06-01 06:06:16,131] Trial 0 finished with value: 1.0845410028997666 and parameters: {'depth': 9, 'l2_leaf_reg': 20.26222771785787, 'learning_rate': 0.030139101566312484, 'random_strength': 0.47224929160909696, 'min_child_samples': 16, 'colsample_bylevel': 0.015148378714587117}. Best is trial 0 with value: 1.0845410028997666.


/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")
/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


...

[I 2023-06-01 06:07:20,145] Trial 493 pruned. Trial was pruned at iteration 16.
[I 2023-06-01 06:07:20,148] Trial 494 pruned. Trial was pruned at iteration 16.


/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")
/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


[I 2023-06-01 06:07:20,455] Trial 496 pruned. Trial was pruned at iteration 16.
[I 2023-06-01 06:07:20,457] Trial 495 pruned. Trial was pruned at iteration 16.


/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")
/tmp/ipykernel_24/3309391134.py:36: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "RMSE")


[I 2023-06-01 06:07:20,760] Trial 497 pruned. Trial was pruned at iteration 16.
[I 2023-06-01 06:07:20,761] Trial 498 pruned. Trial was pruned at iteration 16.
[I 2023-06-01 06:07:20,920] Trial 499 pruned. Trial was pruned at iteration 16.
{'depth': 9, 'l2_leaf_reg': 0.9721028548104323, 'learning_rate': 0.2663736573552741, 'random_strength': 0.2768680192406874, 'min_child_samples': 32, 'colsample_bylevel': 0.07845720822436843}
MSE:  loss_A: 0.42242414334528017
MSE:  loss_B: 1.5834495396271744


# Second case: Training using all of the data from Dataset A

So far we have used 75% of Dataset A to train the data and 25% to test it.
Here, we used all of the data from Dataset A to train, and then test it on Dataset B.

In [12]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import BayesianRidge
from catboost import CatBoostRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
import lightgbm as lgb
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

rf_reg = RandomForestRegressor(n_jobs = -1, max_depth = 50)
# svm_reg = SVR(kernel='linear')
# knn_reg = KNeighborsRegressor(n_neighbors=3)
xgb_reg = XGBRegressor(learning_rate=0.01, n_estimators=300, max_depth=3, subsample=0.7)
#mlp_reg = MLPRegressor(hidden_layer_sizes=(100,), max_iter=10000)
ada_reg = AdaBoostRegressor(n_estimators=50, learning_rate=0.003)
#lr_reg = LinearRegression()
#bay_reg = BayesianRidge()
cat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,
                           depth=6, loss_function='RMSE', random_seed=11)
#pls_reg = PLSRegression(n_components=2)
#rig_reg = Ridge(alpha=1.0)
lgb_reg = lgb.LGBMRegressor()
#el_reg = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=0)
'''bag_reg = BaggingRegressor(
    DecisionTreeRegressor(), n_estimators=50, max_samples=100,
    max_features=1.0,
    bootstrap=True,
    n_jobs=-1)'''

voting_reg = VotingRegressor(
  estimators=[#('nn', mlp_reg),
              #('svc', svm_reg),
              #('knn', knn_reg), 
              ('ada', ada_reg),#('by', bay_reg),
              ('xgb', xgb_reg),('cat', cat_reg)
              #('rf', rf_reg),
              #('el', el_reg), ('lgb', lgb_reg)
             ])
voting_reg.fit(X_A, y_A)

VotingRegressor(estimators=[('ada', AdaBoostRegressor(learning_rate=0.003)),
                            ('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=...
                                          interaction_constraints=None,
                                          learning_rate=0.01, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=3,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=300, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x7e2abc6456c0>)])

In [13]:
from sklearn.metrics import mean_squared_error
for clf in (ada_reg,
            xgb_reg, rf_reg,
            cat_reg, voting_reg):
    clf.fit(X_A[useful_features7], y_A)
    y_pred_B = clf.predict(X_B[useful_features7])
    print(clf.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

AdaBoostRegressor for dataset B: 2.1222028061409035
XGBRegressor for dataset B: 1.684950589268942
RandomForestRegressor for dataset B: 1.617960045184432
CatBoostRegressor for dataset B: 1.7155210834011552
VotingRegressor for dataset B: 1.7644819046850058


In [14]:
xgb_reg.fit(X_train_A[useful_features7], np.array(y_train_A).ravel())
y_pred_B = xgb_reg.predict(X_B[useful_features7])
print(xgb_reg.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

XGBRegressor for dataset B: 1.692358673806465


In [15]:
cat_reg.fit(X_A[useful_features7], y_A)
y_pred_B = cat_reg.predict(X_B[useful_features7])
print(cat_reg.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

CatBoostRegressor for dataset B: 1.7155210834011552


In [16]:
cat_reg = CatBoostRegressor(iterations=500, learning_rate=0.1, verbose = 0,
                           depth=7, loss_function='RMSE', random_seed=11)
cat_reg.fit(X_A[useful_features7], y_A)
y_pred_B = cat_reg.predict(X_B[useful_features7])
print(cat_reg.__class__.__name__ + " for dataset B:", mean_squared_error(y_B, y_pred_B))

CatBoostRegressor for dataset B: 1.665708286683459


In [17]:
X_train_A

,H2,Methane,Acetylene,Ethylene,Ethane,Water,Acid,BDV,IFT
503,19.0,11.90,0.0,2.7,3.0,5,0.005,73.0,30
679,13.2,9.20,0.0,0.6,2.0,4,0.005,88.0,41
240,14.4,2.50,0.0,0.9,1.5,4,0.005,79.0,31
21,13.2,1.10,0.0,1.8,2.1,4,0.024,78.0,25
56,20.9,0.00,0.6,4.2,3.8,6,0.056,76.0,18
...,...,...,...,...,...,...,...,...,...
269,13.7,5.10,0.0,0.4,1.1,1,0.005,94.0,36
337,32.9,3.77,0.0,0.6,2.4,6,0.005,79.0,32
91,22.8,3.30,0.0,4.9,3.0,11,0.140,88.0,16
80,61.2,27.30,0.0,25.6,20.8,9,0.099,70.0,17


# Blending Model

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

# Train the base models
lr = LinearRegression()
rf = RandomForestRegressor()
xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=0)
svm = SVR()
knn = KNeighborsRegressor()
mlp = MLPRegressor(max_iter=5000)

lr.fit(X_train_A, y_train_A)
rf.fit(X_train_A, y_train_A)
xgb.fit(X_train_A, y_train_A)
cat.fit(X_train_A, y_train_A)
svm.fit(X_train_A, y_train_A)
knn.fit(X_train_A, y_train_A)
mlp.fit(X_train_A, y_train_A)

# Make predictions on the test set A
lr_predictions = lr.predict(X_test_A)
rf_predictions = rf.predict(X_test_A)
xgb_predictions = xgb.predict(X_test_A)
cat_predictions = cat.predict(X_test_A)
svm_predictions = svm.predict(X_test_A)
knn_predictions = knn.predict(X_test_A)
mlp_predictions = mlp.predict(X_test_A)

lr_predictions_B = lr.predict(X_B)
rf_predictions_B = rf.predict(X_B)
xgb_predictions_B = xgb.predict(X_B)
cat_predictions_B = cat.predict(X_B)
svm_predictions_B = svm.predict(X_B)
knn_predictions_B = knn.predict(X_B)
mlp_predictions_B = mlp.predict(X_B)

# Combine the predictions using a meta model
meta_model = LinearRegression()
meta_model.fit(np.column_stack([lr_predictions, rf_predictions, xgb_predictions, cat_predictions, svm_predictions, knn_predictions, mlp_predictions]), y_test_A)

# Make predictions on the test set using the meta model
blend_predictions = meta_model.predict(np.column_stack([lr_predictions, rf_predictions, xgb_predictions, cat_predictions, svm_predictions, knn_predictions, mlp_predictions]))
blend_predictions_B = meta_model.predict(np.column_stack([lr_predictions_B, rf_predictions_B, xgb_predictions_B, cat_predictions_B, svm_predictions_B, knn_predictions_B, mlp_predictions_B]))


# Evaluate the model
print('Mean absolute error A:', np.mean(np.abs(blend_predictions - y_test_A)))
print('Mean squared error A:', np.mean((blend_predictions - y_test_A)**2))

print('Mean absolute error B: ', np.mean(np.abs(blend_predictions_B - y_B)))
print('Mean squared error B:', np.mean((blend_predictions_B - y_B)**2))

Mean absolute error A: 0.28096284131820154
Mean squared error A: 0.3425530943316826
Mean absolute error B:  0.6068312322139732
Mean squared error B: 1.739010359139329


In [19]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor

# Set random seed
np.random.seed(42)

# Train the base models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
cat = CatBoostRegressor(verbose=0, random_state=42)
svm = SVR()
knn = KNeighborsRegressor()
mlp = MLPRegressor(max_iter=5000, random_state=42)
el = ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42)
ada = AdaBoostRegressor(random_state=42)

lr.fit(X_train_A[useful_features7], y_train_A)
rf.fit(X_train_A[useful_features7], y_train_A)
xgb.fit(X_train_A[useful_features7], y_train_A)
cat.fit(X_train_A[useful_features7], y_train_A)
svm.fit(X_train_A[useful_features7], y_train_A)
knn.fit(X_train_A[useful_features7], y_train_A)
mlp.fit(X_train_A[useful_features7], y_train_A)
el.fit(X_train_A[useful_features7], y_train_A)
ada.fit(X_train_A[useful_features7], y_train_A)

# Make predictions on the test set A
lr_predictions = lr.predict(X_test_A[useful_features7])
rf_predictions = rf.predict(X_test_A[useful_features7])
xgb_predictions = xgb.predict(X_test_A[useful_features7])
cat_predictions = cat.predict(X_test_A[useful_features7])
svm_predictions = svm.predict(X_test_A[useful_features7])
knn_predictions = knn.predict(X_test_A[useful_features7])
el_predictions = el.predict(X_test_A[useful_features7])
ada_predictions = ada.predict(X_test_A[useful_features7])

lr_predictions_B = lr.predict(X_B[useful_features7])
rf_predictions_B = rf.predict(X_B[useful_features7])
xgb_predictions_B = xgb.predict(X_B[useful_features7])
cat_predictions_B = cat.predict(X_B[useful_features7])
svm_predictions_B = svm.predict(X_B[useful_features7])
knn_predictions_B = knn.predict(X_B[useful_features7])
mlp_predictions_B = mlp.predict(X_B[useful_features7])
el_predictions_B = el.predict(X_B[useful_features7])
ada_predictions_B = ada.predict(X_B[useful_features7])

# Combine the predictions using a meta model
meta_model = ElasticNet(alpha=0.01, l1_ratio=0.01, random_state=42)
meta_model.fit(np.column_stack([lr_predictions, rf_predictions, xgb_predictions, cat_predictions, svm_predictions, knn_predictions, ada_predictions
                                ,el_predictions]), y_test_A)

# Make predictions on the test set using the meta model
blend_predictions = meta_model.predict(np.column_stack([lr_predictions, rf_predictions, xgb_predictions, cat_predictions, svm_predictions, knn_predictions, ada_predictions
                                                        , el_predictions]))
blend_predictions_B = meta_model.predict(np.column_stack([lr_predictions_B, rf_predictions_B, xgb_predictions_B, cat_predictions_B, svm_predictions_B, knn_predictions_B, ada_predictions_B
                                                          , el_predictions_B]))


# Evaluate the model
print('Mean absolute error A:', np.mean(np.abs(blend_predictions - y_test_A)))
print('Mean squared error A:', np.mean((blend_predictions - y_test_A)**2))

print('Mean absolute error B: ', np.mean(np.abs(blend_predictions_B - y_B)))
print('Mean squared error B:', np.mean((blend_predictions_B - y_B)**2))


Mean absolute error A: 0.27378772372364946
Mean squared error A: 0.30416806747629016
Mean absolute error B:  0.4171020179089863
Mean squared error B: 1.5122398820868401


In [20]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor

# Set random seed
np.random.seed(42)

# Train the base models
lr = LinearRegression()
rf = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
cat = CatBoostRegressor(verbose=0, random_state=42)
svm = SVR()
knn = KNeighborsRegressor()
mlp = MLPRegressor(max_iter=5000, random_state=42)
el = ElasticNet(alpha=0.1, l1_ratio=0.1, random_state=42)
ada = AdaBoostRegressor(random_state=42)

lr.fit(X_train_A[useful_features7], y_train_A)
rf.fit(X_train_A[useful_features7], y_train_A)
xgb.fit(X_train_A[useful_features7], y_train_A)
cat.fit(X_train_A[useful_features7], y_train_A)
svm.fit(X_train_A[useful_features7], y_train_A)
knn.fit(X_train_A[useful_features7], y_train_A)
mlp.fit(X_train_A[useful_features7], y_train_A)
el.fit(X_train_A[useful_features7], y_train_A)
ada.fit(X_train_A[useful_features7], y_train_A)

# Make predictions on the test set A
lr_predictions = lr.predict(X_test_A[useful_features7])
rf_predictions = rf.predict(X_test_A[useful_features7])
xgb_predictions = xgb.predict(X_test_A[useful_features7])
cat_predictions = cat.predict(X_test_A[useful_features7])
svm_predictions = svm.predict(X_test_A[useful_features7])
knn_predictions = knn.predict(X_test_A[useful_features7])
el_predictions = el.predict(X_test_A[useful_features7])
ada_predictions = ada.predict(X_test_A[useful_features7])

lr_predictions_B = lr.predict(X_B[useful_features7])
rf_predictions_B = rf.predict(X_B[useful_features7])
xgb_predictions_B = xgb.predict(X_B[useful_features7])
cat_predictions_B = cat.predict(X_B[useful_features7])
svm_predictions_B = svm.predict(X_B[useful_features7])
knn_predictions_B = knn.predict(X_B[useful_features7])
mlp_predictions_B = mlp.predict(X_B[useful_features7])
el_predictions_B = el.predict(X_B[useful_features7])
ada_predictions_B = ada.predict(X_B[useful_features7])

# Combine the predictions using a meta model
meta_model = AdaBoostRegressor(random_state=42)
meta_model.fit(np.column_stack([lr_predictions, rf_predictions, xgb_predictions, cat_predictions, svm_predictions, knn_predictions, ada_predictions
                                ,el_predictions]), y_test_A)

# Make predictions on the test set using the meta model
blend_predictions = meta_model.predict(np.column_stack([lr_predictions, rf_predictions, xgb_predictions, cat_predictions, svm_predictions, knn_predictions, ada_predictions
                                                        , el_predictions]))
blend_predictions_B = meta_model.predict(np.column_stack([lr_predictions_B, rf_predictions_B, xgb_predictions_B, cat_predictions_B, svm_predictions_B, knn_predictions_B, ada_predictions_B
                                                          , el_predictions_B]))


# Evaluate the model
print('Mean absolute error A:', np.mean(np.abs(blend_predictions - y_test_A)))
print('Mean squared error A:', np.mean((blend_predictions - y_test_A)**2))

print('Mean absolute error B: ', np.mean(np.abs(blend_predictions_B - y_B)))
print('Mean squared error B:', np.mean((blend_predictions_B - y_B)**2))


Mean absolute error A: 0.3540009165335157
Mean squared error A: 0.1399913740114584
Mean absolute error B:  0.6144019400215026
Mean squared error B: 1.3691966716301185
